In [1]:
pip install git+https://github.com/datakaveri/iudx-python-sdk

  Cloning https://github.com/datakaveri/iudx-python-sdk to c:\users\rajha\appdata\local\temp\pip-req-build-9eqf44hh
  Resolved https://github.com/datakaveri/iudx-python-sdk to commit 82c04ef5cc78338f43d7d34fd17d860bb39a5865
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/datakaveri/iudx-python-sdk 'C:\Users\rajha\AppData\Local\Temp\pip-req-build-9eqf44hh'


In [3]:
import pyIUDX
import pandas as pd
import numpy as np
from datetime import datetime

from tqdm import tqdm

from scipy.interpolate import griddata

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from iudx.entity.Entity import Entity
from iudx.auth.Token import Token

import folium
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import geojsoncontour

import ipywidgets as widgets
from ipywidgets import Layout

import warnings
warnings.filterwarnings('ignore')
from fastparquet import ParquetFile

In [4]:
#Get token from backend, this is automatic in the sandbox environment
#token = Token(auth_url="https://authorization.iudx.org.in/auth/v1/token", token_file='/tmp/credentials.json')

"""
If running locally, you need to provide your client credentials as shown below
"""
client_id=input("client id : 6371b5f1-5c0d-4a3b-8cc9-5aa90a0aec1b")
client_secret=input("client secret : 69f4f4bc1000e5cf03c7a78720dba709e978d9c1")
token = Token(client_id=client_id, client_secret=client_secret)



token.set_item(
    item_id="rs.iudx.org.in",
    item_type="resource_server",
    role="consumer")

# Resource group ID for Pune AQM sensors
item_id="datakaveri.org/04a15c9960ffda227e9546f3f46e629e1fe4132b/rs.iudx.org.in/pune-env-aqm"

# Set a private resource to access.
token.set_item(
    item_id=item_id,
    item_type="resource_group",
    role="consumer")

# Creating an entity for the resource-group (use the token parameter to access private resources)
# Make sure all the resources in the group have the same schema (Ex. Pune AQM)
entity = Entity(item_id, token_obj=token)

#create a during query for a time interval.
#aqm = entity.during_search(
#          start_time="2022-02-22T00:00:00Z",
#          end_time="2022-03-22T00:00:00Z"
#      )

6371b5f1-5c0d-4a3b-8cc9-5aa90a0aec1b6371b5f1-5c0d-4a3b-8cc9-5aa90a0aec1b
69f4f4bc1000e5cf03c7a78720dba709e978d9c169f4f4bc1000e5cf03c7a78720dba709e978d9c1


In [18]:
# Read data
# Datasets for other time periods can be found on the Sandbox page here
# https://sandbox.iudx.org.in/dataset/datakaveri.org-f7e044eee8122b5c87dce6e7ad64f3266afa41dc-cat-sandbox.iudx.org.in-pune-env-aqm

aqm=pd.read_parquet('https://iudx-cat-sandbox-dev.s3.ap-south-1.amazonaws.com/pune-aqm-data/pune-aqm-feb.parquet', engine='fastparquet')
aqm.drop_duplicates()
aqm.columns
aqm.head()


,airQualityLevel,observationDateTime,deviceStatus,aqiMajorPollutant,id,airQualityIndex,uv.avgOverTime,o3.avgOverTime,pm2p5.avgOverTime,co2.avgOverTime,pm10.avgOverTime,co.avgOverTime,no2.avgOverTime,airTemperature.avgOverTime,illuminance.avgOverTime,ambientNoise.avgOverTime,so2.avgOverTime,relativeHumidity.avgOverTime,atmosphericPressure.avgOverTime
0,SATISFACTORY,2022-01-31 23:31:25+05:00,ACTIVE,CO,datakaveri.org/04a15c9960ffda227e9546f3f46e629...,71.0,0.0,32.97,16.94,322.99,35.48,3.04,78.51,21.30,4.52,74.53,0.06,62.28,0.46
1,SATISFACTORY,2022-01-31 23:31:25+05:00,ACTIVE,NO2,datakaveri.org/04a15c9960ffda227e9546f3f46e629...,99.0,0.0,1.21,48.09,525.40,82.88,1.32,125.37,23.29,3.23,76.57,2.42,58.05,0.94
2,SATISFACTORY,2022-01-31 23:31:25+05:00,ACTIVE,NO2,datakaveri.org/04a15c9960ffda227e9546f3f46e629...,81.0,0.0,14.09,32.27,424.46,76.27,1.32,85.96,21.65,24.61,71.48,0.24,55.42,0.93
3,SATISFACTORY,2022-01-31 23:31:25+05:00,ACTIVE,NO2,datakaveri.org/04a15c9960ffda227e9546f3f46e629...,97.0,0.0,24.03,47.48,162.78,94.92,1.14,97.11,22.06,1.39,84.27,4.80,60.90,0.89
4,SATISFACTORY,2022-01-31 23:31:25+05:00,ACTIVE,CO,datakaveri.org/04a15c9960ffda227e9546f3f46e629...,72.0,0.0,30.78,58.04,0.00,96.20,1.83,75.28,22.36,2.89,81.63,4.30,62.43,0.94


In [22]:
rsc_dict={}
for rsc in entity.resources:
    rsc_dict[rsc['id']]={
        'coordinates':rsc['location']['geometry']['coordinates'],
        'label':rsc['label']
    }

In [23]:
sensors_map=folium.Map()
for rsc_id, rsc in rsc_dict.items():
    folium.Marker([rsc["coordinates"][1], rsc["coordinates"][0]],
                  tooltip=rsc['label']).add_to(sensors_map)
sensors_map.fit_bounds(sensors_map.get_bounds())
sensors_map

In [24]:
# adding sensor location and label information to the dataframe
aqm['latitude']=aqm['id'].apply(lambda id:rsc_dict[id]['coordinates'][1])
aqm['longitude']=aqm['id'].apply(lambda id:rsc_dict[id]['coordinates'][0])
aqm['label']=aqm['id'].apply(lambda id:rsc_dict[id]['label'])

In [25]:
column_choice=['airTemperature','ambientNoise','so2','uv','co','co2','atmosphericPressure',
               'illuminance','no2','o3','pm10','pm2p5','relativeHumidity']

In [26]:
prompt1=widgets.HTML(value="")
prompt2=widgets.HTML(value="")
gif_address = 'https://www.uttf.com.ua/assets/images/loader2.gif'

# drop down widget to choose pollutant/property
select_col=widgets.Dropdown(
    options=column_choice,
    value='pm10',
    description='Property:',
    disabled=False,
)

time_range=pd.date_range(datetime(2022,3,21,0), datetime(2022,3,22,0), freq='H')
options = [time.strftime('%d %b %H:%M:%S') for time in time_range]
index = (0, len(options)-1)

# Slider to choose time
select_time=widgets.SelectionSlider(
    options=options,
    value=options[-1],
    description='Time:',
    disabled=False,
    orientation='horizontal',
    continuous_update=False,
    readout=True,
    width=1000,
)
mywidgets=[select_col]
ui=widgets.VBox([prompt1,select_col,select_time,prompt2],width=500)

In [27]:
# FUNCTION TO PLOT CONTOUR MAP AFTER CUBIC INTERPOLATION
def pollutant_contour(pollutant,time):
    prompt2.value='Building Contour ...'
    
    # GATHER THE REQUIRED DATA FOT THE CHOSEN TIME
    time=datetime.strptime(time+' 2022','%d %b %H:%M:%S %Y')
    aqm_latest=aqm[aqm.observationDateTime<time.strftime('%d-%m-%Y %H:%M:%S')]
    aqm_latest=aqm_latest.sort_values(by='observationDateTime').drop_duplicates(['id'],keep='last')

    pollutant=pollutant+'.avgOverTime'
    data=aqm_latest.dropna(subset=[pollutant])
    readings=data[pollutant]
    x=data['longitude']
    y=data['latitude']
    sensors=list(np.unique(data.id))
    
    #BUILDING A MESH GRID FOR A GIVEN RESOLUTION
    res=500
    y_arr = np.linspace(np.min(y), np.max(y), res)
    x_arr = np.linspace(np.min(x), np.max(x), res)
    x_mesh, y_mesh = np.meshgrid(x_arr, y_arr)
    
    # PERFORM CUBIC INTERPOLATION
    interpolated_readings = griddata((x, y), readings, (x_mesh, y_mesh), method='cubic')
    
    # PLOT CONTOUR
    levels = 50
    contourf=plt.contourf(x_mesh, y_mesh, interpolated_readings, levels, alpha=0.5, 
                          cmap="bwr", linestyles='None', vmin=0)
    plt.close()

    # CONVERT CONTOUR TO GEOJSON REPRESENTION
    geojson = geojsoncontour.contourf_to_geojson(
        contourf=contourf,
        min_angle_deg=3.0,
        ndigits=5,
        stroke_width=1,
        fill_opacity=0.5)

    sensors_map=folium.Map(tiles="cartodbpositron")
    
    # UNCOMMENT THE FOLLOWING LINES TO PLOT THE SENSOR LOCATIONS AS WELL
    # for rsc_id, rsc in rsc_dict.items():
    #     if rsc_id in sensors:
    #         folium.Marker([rsc["coordinates"][1], rsc["coordinates"][0]],
    #                       tooltip=rsc['label']).add_to(sensors_map)

    # PLOT CONTOUR ON MAP WITH FOLIUM 
    folium.GeoJson(
        geojson,
        style_function=lambda x: {
            'color':     x['properties']['stroke'],
            'weight':    x['properties']['stroke-width'],
            'fillColor': x['properties']['fill'],
            'opacity':   0.6,
        }).add_to(sensors_map)

    sensors_map.fit_bounds(sensors_map.get_bounds())
    
    prompt2.value=str(time)
    
    display(sensors_map)

In [28]:
display(ui)
widgets.interactive_output(pollutant_contour,{'pollutant':select_col,'time':select_time})

Output()